In [47]:
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm.auto import tqdm

import json
import minsearch
import openai
import os

In [48]:
client = OpenAI()

In [49]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": "What your name?"}]
)

In [50]:
response.choices[0].message.content

"I'm an artificial intelligence created by OpenAI, often referred to as ChatGPT. How can I assist you today?"

## Retrivel and Search

In [6]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 15:14:35--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642,90K  3,90MB/s    in 0,2s    

2024-06-28 15:14:36 (3,90 MB/s) - ‘documents.json’ saved [658332/658332]



In [51]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [52]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


In [53]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [54]:
index.fit(documents)

In [55]:
q = 'the course has already started. Can I still enroll?'

In [56]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [57]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generating Answers with OpenAI GPT 4.0

In [58]:
q

'the course has already started. Can I still enroll?'

In [59]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": q}]
)

response.choices[0].message.content

"It's common for institutions and online platforms to have different policies regarding late enrollment. Here are a few steps you can take:\n\n1. **Check the Official Policy**: Visit the course or institution's website to see if they have a policy on late enrollment. Some courses allow late sign-ups, sometimes with a deadline a week or two after the course begins.\n\n2. **Contact the Instructor or Administration**: Reach out directly to the course instructor or the administrative office. They might be willing to make an exception or provide you with alternative solutions.\n\n3. **Catch Up on Missed Material**: If you are allowed to enroll late, make sure to review any materials you missed. Many online courses provide recordings of past lectures and other resources that can help you get up to speed.\n\n4. **Consider Upcoming Sessions**: If late enrollment isn’t possible, ask if the course will be offered again in the near future. This way, you can plan to enroll at the next opportunity.

In [60]:
prompt_template = '''
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only facts from the CONTEXT when answering the qUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}

'''.strip()

In [61]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [62]:
prompt = prompt_template.format(question=q, context=context)
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only facts from the CONTEXT when answering the qUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started. Can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final

In [63]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, even if the course has already started, you can still join and are eligible to submit the homework. Be aware of the deadlines for turning in the final projects.'

## The RAG Flow Cleaning and Modularizing Code

In [64]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [65]:
def build_prompt(query, search_results):
    prompt_template = '''
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT: {context}
    
    '''.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [66]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [67]:
def rag(query):
    results = search(query)
    builded_prompt = build_prompt(query, results)
    answer = llm(builded_prompt)

    return answer

In [68]:
query = "the course has already started. Can I still enroll?"
print(rag(query))

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [69]:
query = "How do I run kafka?"
print(rag(query))

NONE


## Elastic Search

In [6]:
!curl http://localhost:9200

{
  "name" : "ecd5e09273f1",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "j4HB0nflRD6CzmRFS60UVw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
es_client.info()

ObjectApiResponse({'name': 'ecd5e09273f1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'j4HB0nflRD6CzmRFS60UVw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████| 948/948 [00:10<00:00, 88.38it/s]


In [70]:
query = "I just discovered the course, Can I still join it?"

In [71]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [73]:
def rag(query):
    results = elastic_search(query)
    builded_prompt = build_prompt(query, results)
    answer = llm(builded_prompt)

    return answer

In [74]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."